In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

In [3]:
import random

In [5]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [6]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [7]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

41 documents
8 classes ['answer', 'goodbye', 'greeting', 'more', 'noanswer', 'options', 'surprise', 'thanks']
54 unique lemmatized words ["'s", ',', 'a', 'anyone', 'are', 'awesome', 'be', 'bye', 'can', 'chatting', 'could', 'day', 'do', 'fact', 'for', 'fsdh', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'is', 'later', 'lookup', 'me', 'next', 'nice', 'offered', 'provide', 'random', 'see', 'sjdfhk', 'support', 'surprise', 'tell', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'wa', 'what', 'when', 'where', 'who', 'y', 'you']


In [8]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [9]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")
print("X = ", train_x)
print()
print()
print("Y = ", train_y)


Training data created
X =  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0

<ipython-input-9-69f8db72295d>:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [10]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [11]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [12]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=400, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

Epoch 1/400
9/9 [==============================] - 1s 5ms/step - loss: 2.0905 - accuracy: 0.1448
Epoch 2/400
9/9 [==============================] - 0s 2ms/step - loss: 2.1208 - accuracy: 0.1146
Epoch 3/400
9/9 [==============================] - 0s 2ms/step - loss: 1.9965 - accuracy: 0.1750
Epoch 4/400
9/9 [==============================] - 0s 1ms/step - loss: 1.9005 - accuracy: 0.3453
Epoch 5/400
9/9 [==============================] - 0s 2ms/step - loss: 1.8121 - accuracy: 0.3740
Epoch 6/400
9/9 [==============================] - 0s 1ms/step - loss: 1.8127 - accuracy: 0.2256
Epoch 7/400
9/9 [==============================] - 0s 1ms/step - loss: 1.7314 - accuracy: 0.3504
Epoch 8/400
9/9 [==============================] - 0s 1ms/step - loss: 1.6427 - accuracy: 0.4303
Epoch 9/400
9/9 [==============================] - 0s 2ms/step - loss: 1.6291 - accuracy: 0.3376
Epoch 10/400
9/9 [==============================] - 0s 2ms/step - loss: 1.5511 - accuracy: 0.5067
Epoch 11/400
9/9 [===========

9/9 [==============================] - 0s 1ms/step - loss: 0.0731 - accuracy: 0.9926
Epoch 85/400
9/9 [==============================] - 0s 1ms/step - loss: 0.1074 - accuracy: 0.9824
Epoch 86/400
9/9 [==============================] - 0s 1ms/step - loss: 0.0490 - accuracy: 0.9951
Epoch 87/400
9/9 [==============================] - 0s 999us/step - loss: 0.0801 - accuracy: 0.9824
Epoch 88/400
9/9 [==============================] - 0s 1ms/step - loss: 0.0834 - accuracy: 0.9926
Epoch 89/400
9/9 [==============================] - 0s 1ms/step - loss: 0.1843 - accuracy: 0.9308
Epoch 90/400
9/9 [==============================] - 0s 1ms/step - loss: 0.1107 - accuracy: 0.9824
Epoch 91/400
9/9 [==============================] - 0s 1ms/step - loss: 0.0598 - accuracy: 1.0000
Epoch 92/400
9/9 [==============================] - 0s 1ms/step - loss: 0.0511 - accuracy: 1.0000
Epoch 93/400
9/9 [==============================] - 0s 1ms/step - loss: 0.1499 - accuracy: 0.9334
Epoch 94/400
9/9 [=============

9/9 [==============================] - 0s 1ms/step - loss: 0.0613 - accuracy: 1.0000
Epoch 167/400
9/9 [==============================] - 0s 1ms/step - loss: 0.0247 - accuracy: 1.0000
Epoch 168/400
9/9 [==============================] - 0s 2ms/step - loss: 0.0397 - accuracy: 0.9926
Epoch 169/400
9/9 [==============================] - 0s 1ms/step - loss: 0.0273 - accuracy: 1.0000
Epoch 170/400
9/9 [==============================] - 0s 1ms/step - loss: 0.0763 - accuracy: 0.9608
Epoch 171/400
9/9 [==============================] - 0s 914us/step - loss: 0.0278 - accuracy: 1.0000
Epoch 172/400
9/9 [==============================] - 0s 1ms/step - loss: 0.0303 - accuracy: 1.0000
Epoch 173/400
9/9 [==============================] - 0s 1ms/step - loss: 0.0149 - accuracy: 1.0000
Epoch 174/400
9/9 [==============================] - 0s 1ms/step - loss: 0.0302 - accuracy: 1.0000
Epoch 175/400
9/9 [==============================] - 0s 1ms/step - loss: 0.0478 - accuracy: 1.0000
Epoch 176/400
9/9 [===

9/9 [==============================] - 0s 1ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 249/400
9/9 [==============================] - 0s 1ms/step - loss: 0.0173 - accuracy: 1.0000
Epoch 250/400
9/9 [==============================] - 0s 1ms/step - loss: 0.0151 - accuracy: 1.0000
Epoch 251/400
9/9 [==============================] - 0s 1ms/step - loss: 0.0334 - accuracy: 1.0000
Epoch 252/400
9/9 [==============================] - 0s 2ms/step - loss: 0.0068 - accuracy: 1.0000
Epoch 253/400
9/9 [==============================] - 0s 2ms/step - loss: 0.0369 - accuracy: 1.0000
Epoch 254/400
9/9 [==============================] - 0s 969us/step - loss: 0.0090 - accuracy: 1.0000
Epoch 255/400
9/9 [==============================] - 0s 1ms/step - loss: 0.0197 - accuracy: 1.0000
Epoch 256/400
9/9 [==============================] - 0s 1ms/step - loss: 0.0201 - accuracy: 1.0000
Epoch 257/400
9/9 [==============================] - 0s 1ms/step - loss: 0.0162 - accuracy: 1.0000
Epoch 258/400
9/9 [===

9/9 [==============================] - 0s 1ms/step - loss: 0.0176 - accuracy: 1.0000
Epoch 331/400
9/9 [==============================] - 0s 1ms/step - loss: 0.0177 - accuracy: 1.0000
Epoch 332/400
9/9 [==============================] - 0s 1ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 333/400
9/9 [==============================] - 0s 2ms/step - loss: 0.0059 - accuracy: 1.0000
Epoch 334/400
9/9 [==============================] - 0s 999us/step - loss: 0.0023 - accuracy: 1.0000
Epoch 335/400
9/9 [==============================] - 0s 954us/step - loss: 0.0039 - accuracy: 1.0000
Epoch 336/400
9/9 [==============================] - 0s 1ms/step - loss: 0.0068 - accuracy: 1.0000
Epoch 337/400
9/9 [==============================] - 0s 1ms/step - loss: 0.0249 - accuracy: 1.0000
Epoch 338/400
9/9 [==============================] - 0s 1ms/step - loss: 0.0133 - accuracy: 1.0000
Epoch 339/400
9/9 [==============================] - 0s 1ms/step - loss: 0.0134 - accuracy: 1.0000
Epoch 340/400
9/9 [=